In [108]:
import re
from gurobipy import *
import numpy as np
import itertools
import time
import linecache as lc
import pandas as pd

In [109]:
def read_file(filename):
    '''
    Reads problem data into arrays that can be used to solve the problem
    '''
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrixMetres']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

def allocation_from_txt(file):
    '''
    Reads an allocation vector from a txt file to an appropriate array for the local search heuristic and other purposes.
    '''
    allocation_vector = lc.getline(file, 1).split(" ")[:-1]
    allocation_vector = [int(prod) for prod in allocation_vector]
    return np.array(allocation_vector)

# Reading in data and running the entire function

In [110]:
filename = r"Xpress_Data_Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrixMetres'))
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)

# Initialise number of shelves and shelf numbers
NbShelves = 96
Shelves = range(1, NbShelves + 1)


# Calculating total distance from an allocation

In [111]:
def find_closest_product(current_shelf, order, distances):
    '''
    Find the closest shelf which contains a product in the order from the current shelf
    '''
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf

In [138]:
def q1_function(allocation_vector, distance_matrix):
    '''
    Calculates the total walking distance to pick products from shelves for all orders. 
    Also returns the distances covered in each order, the distances per order in decreasing order, 
    the indices and routes corresponding to each order where the distances are decreasing.
    '''
    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, distance_matrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

    # List of routes ordered by distance (descending)
    OrderedRoutes = [routes[i] for i in idx_longest_orders]
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

# Local Search

In [146]:
def find_new_shelf_prod(shelf, allocations):
    '''
    Finds the shelf number and product on the shelf opposite to the provided shelf 
    '''
    if (shelf <= 24) or (49 <= shelf <= 72): 
        new_shelf = shelf + 24 # opposite shelf
    elif (25 <= shelf <= 48) or (73 <= shelf <= 96):
        new_shelf = shelf - 24 # opposite shelf
    new_prod = allocations[new_shelf-1] # product on the opposite shelf

    return new_prod, new_shelf

In [151]:
def swap_prods_worst_route_neighbourhoods(initial_allocation, max_ind, distance_matrix):
    '''
    Create neighbourhood structures by looking at each order in decreasing order of distance covered in the order.
    Within each of these orders, fix a product and shelf and move another product in the order to a shelf opposite the fixed shelf and product.
    Accept the neighbourhood if it decreases the total walking distance in comparison to the initial allocation.
    max_ind decides how many orders we examine to introduce the neighbourhoods to. 
    max_ind usually takes a value of 5 or 10 to look at the 10 longest orders in terms of distance covered.
    '''
    start_time = time.time()
    init_alloc = initial_allocation.copy()
    orders_shelf = convert_orders_to_shelf_indices(init_alloc)
    init_result = q1_function(init_alloc, distance_matrix)
    init_distance, sorted_dist, idx_sorted_dist, sorted_routes = init_result[0], init_result[2], init_result[3], init_result[4]
    neighbourhoods = []
    neigh_dists = []
    print(f"Initial distance : {init_distance}")

    # Loops over the orders with the highest distances walked up to kth longest distance orders
    for k_ind, k in enumerate(idx_sorted_dist[:max_ind+1]):
        # Finds the products and shelves corresponding to the order
        prods = Orders[k]
        shelfs = orders_shelf[k]
        
        # If there is more than one shelf containing the product in the order, choose the shelf covered in the route.
        while any(isinstance(shelf, tuple) for shelf in shelfs):
            chosen_shelf = set(sorted_routes[k_ind]) - set(shelfs) - {0}
            tuple_ind = [isinstance(shelf, tuple) for shelf in shelfs].index(True)
            shelfs[tuple_ind] = chosen_shelf.pop()

        # Removes 0s from the shelfs and product arrays
        shelfs_nonzero = [i for i in shelfs if i != 0]
        prods_nonzero = [i for i in prods if i != 0]

        # Loops over shelves associated with the long distance order
        for i in range(len(shelfs_nonzero)):

            # Fix a product and shelf combination
            p_fix, s_fix = prods_nonzero[i], shelfs_nonzero[i]

            # Create an array of all other product and shelf combinations in the order
            prods_temp, shelfs_temp = prods_nonzero.copy(), shelfs_nonzero.copy()
            prods_temp.remove(p_fix), shelfs_temp.remove(s_fix)

            # Iterate over the remaining products and shelves
            for j in range(len(prods_temp)):

                # Initialise the new allocation
                update_alloc = init_alloc.copy()

                # Choose a product that we will remove from the shelf opposite the fixed shelf and the corresponding shelf of the chosen product
                p_change, s_change = prods_temp[j], shelfs_temp[j]

                # Find the shelf and product opposite fixed shelf
                p_new, s_new = find_new_shelf_prod(s_fix, init_alloc)

                # Exchange the products in the two shelves so that the product in the order goes opposite the fixed shelf
                update_alloc[s_change - 1] = p_new
                update_alloc[s_new - 1] = p_change

                # Results of the updated allocation
                new_result = q1_function(update_alloc, distance_matrix)

                # Add this as a neighbourhood if the total distance covered is lower than the initial distance
                if new_result[0] < init_distance:
                    neighbourhoods.append(update_alloc)
                    neigh_dists.append(new_result[0])


    end_time = time.time()

    return neighbourhoods, np.array(neigh_dists), end_time - start_time

In [152]:
def LSH(allocation, distance_matrix, max_k, time_limit):
    '''
    Local search heuristic which takes the best allocation found so far and explores neighbourhood structures 
    to choose the best neighbourhood structure as the new best allocation. The process repeats till no better 
    solution is found or the time limit has been reached.
    '''
    vns_it = 1
    start_time = time.time()
    init_allocations = allocation.copy()
    best_allocation = init_allocations
    # best_distance = q1_function(best_allocation)[0]
    neighs_to_explore, neigh_distances, neigh_time = swap_prods_worst_route_neighbourhoods(best_allocation, max_k, distance_matrix)
    print(f"Time taken to find neighbourhoods: {neigh_time:.2f} seconds.")
    while time.time() - start_time < time_limit and len(neighs_to_explore) > 0:
        best_ind = np.argmin(neigh_distances)
        # best_dist = neigh_distances[best_ind]
        print(f"===== VNS iteration {vns_it} =====")
        print(f"Time elapsed so far: {time.time() - start_time:.2f} seconds.")
        best_allocation = neighs_to_explore[best_ind]
        neighs_to_explore, neigh_distances, neigh_time = swap_prods_worst_route_neighbourhoods(best_allocation, max_k, distance_matrix)
        print(f"Time taken to find neighbourhoods: {neigh_time:.2f} seconds.")
        vns_it += 1

    return best_allocation, time.time() - start_time

## Running Local Search for CA, MCP, GH and Random

In [116]:
distance_mat_metres = pd.read_excel("Distance_Matrices/DistanceMatrix_0_aisles.xlsx", sheet_name = "DistanceMatrixMetres", header=None, skiprows= 1)

In [118]:
# MCP with 6 extra shelves filled
HeuristicAllocation = allocation_from_txt("Allocation_Vectors/CommonOrder_6_Extra.txt")
print("Results for LSH on the Most Common Product ConstructiveHeuristic")
best_allocation, time_taken = LSH(HeuristicAllocation, distance_matrix = distance_mat_metres, max_k=10, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Most Common Product ConstructiveHeuristic
Initial distance : 203808
Time taken to find neighbourhoods: 183.35 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 183.35 seconds.
Initial distance : 203538
Time taken to find neighbourhoods: 190.78 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 374.14 seconds.
Initial distance : 203466
Time taken to find neighbourhoods: 295.05 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 669.19 seconds.
Initial distance : 203370
Time taken to find neighbourhoods: 249.89 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 919.08 seconds.
Initial distance : 203298
Time taken to find neighbourhoods: 223.15 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 1142.24 seconds.
Initial distance : 202980
Time taken to find neighbourhoods: 279.54 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 1421.95 seconds.
Initial distance : 202896
Time taken to find neighbourhoods: 660.23 seconds.
====

In [119]:
# MCP with 0 extra shelves filled
HeuristicAllocation = allocation_from_txt("Allocation_Vectors/CommonOrder_0_Extra.txt")
print("Results for LSH on the Most Common Product ConstructiveHeuristic")
best_allocation, time_taken = LSH(HeuristicAllocation, distance_matrix = distance_mat_metres, max_k=10, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Most Common Product ConstructiveHeuristic
Initial distance : 205038
Time taken to find neighbourhoods: 185.25 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 185.27 seconds.
Initial distance : 204504
Time taken to find neighbourhoods: 135.93 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 321.20 seconds.
Initial distance : 204342
Time taken to find neighbourhoods: 117.70 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 438.90 seconds.
Initial distance : 204096
Time taken to find neighbourhoods: 103.13 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 542.04 seconds.
Initial distance : 203892
Time taken to find neighbourhoods: 124.56 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 666.60 seconds.
Initial distance : 203754
Time taken to find neighbourhoods: 112.71 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 779.31 seconds.
Initial distance : 203526
Time taken to find neighbourhoods: 107.76 seconds.
===== 

In [121]:
# Current allocation
print("Results for LSH on the Current Allocation")
best_allocation, time_taken = LSH(Allocations.copy(), distance_matrix = distance_mat_metres, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Current Allocation
Initial distance : 290706
Time taken to find neighbourhoods: 142.30 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 142.31 seconds.
Initial distance : 286110
Time taken to find neighbourhoods: 139.72 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 282.02 seconds.
Initial distance : 281142
Time taken to find neighbourhoods: 89.08 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 371.11 seconds.
Initial distance : 276624
Time taken to find neighbourhoods: 87.54 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 458.65 seconds.
Initial distance : 273486
Time taken to find neighbourhoods: 128.78 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 587.43 seconds.
Initial distance : 270738
Time taken to find neighbourhoods: 109.67 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 697.10 seconds.
Initial distance : 268668
Time taken to find neighbourhoods: 86.80 seconds.
===== VNS iteration 7 =====
Time

In [122]:
# Random allocation
RandomAllocation = allocation_from_txt("Allocation_Vectors/Random_6_Extra.txt")
print("Results for LSH on the Random Constructive Heuristic allocation")
best_allocation, time_taken = LSH(RandomAllocation, distance_matrix = distance_mat_metres, max_k=10, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Random Constructive Heuristic allocation
Initial distance : 278652
Time taken to find neighbourhoods: 40.49 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 40.49 seconds.
Initial distance : 274272
Time taken to find neighbourhoods: 46.00 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 86.49 seconds.
Initial distance : 272016
Time taken to find neighbourhoods: 49.71 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 136.20 seconds.
Initial distance : 269466
Time taken to find neighbourhoods: 47.04 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 183.24 seconds.
Initial distance : 267516
Time taken to find neighbourhoods: 48.90 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 232.14 seconds.
Initial distance : 264378
Time taken to find neighbourhoods: 44.26 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 276.40 seconds.
Initial distance : 261582
Time taken to find neighbourhoods: 44.52 seconds.
===== VNS iterat

In [123]:
# Greedy Allocation
GreedyAllocation = allocation_from_txt("Allocation_Vectors/SlightlyGreedy_6_Extra.txt")
print("Results for LSH on the Slightly Greedy Constructive Heuristic allocation")
best_allocation, time_taken = LSH(GreedyAllocation, distance_matrix = distance_mat_metres, max_k=10, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Slightly Greedy Constructive Heuristic allocation
Initial distance : 201888
Time taken to find neighbourhoods: 204.63 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 204.64 seconds.
Initial distance : 201558
Time taken to find neighbourhoods: 227.31 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 431.95 seconds.
Initial distance : 201420
Time taken to find neighbourhoods: 190.79 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 622.74 seconds.
Initial distance : 201414
Time taken to find neighbourhoods: 179.18 seconds.
Total time taken: 801.92 seconds.


# Experimenting different layouts on LSH

## Using current allocation

In [124]:
# 1 middle aisle - current allocation
print("Results for LSH on the Current Allocation with 1 middle aisle")
distance_1_aisle = pd.read_excel("Distance_Matrices/DistanceMatrix_1_aisles.xlsx", sheet_name = "DistanceMatrixMetres", header=None, skiprows= 1)
best_allocation, time_taken = LSH(Allocations.copy(), distance_matrix = distance_1_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Current Allocation with 1 middle aisle
Initial distance : 284820
Time taken to find neighbourhoods: 56.34 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 56.34 seconds.
Initial distance : 277122
Time taken to find neighbourhoods: 53.57 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 109.91 seconds.
Initial distance : 271806
Time taken to find neighbourhoods: 61.93 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 171.84 seconds.
Initial distance : 263046
Time taken to find neighbourhoods: 51.30 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 223.15 seconds.
Initial distance : 259170
Time taken to find neighbourhoods: 48.54 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 271.68 seconds.
Initial distance : 254538
Time taken to find neighbourhoods: 48.94 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 320.62 seconds.
Initial distance : 250506
Time taken to find neighbourhoods: 44.84 seconds.
===== VNS iterati

In [125]:
# 2 middle aisle - current allocation
print("Results for LSH on the Current Allocation with 2 middle aisle")
distance_2_aisle = pd.read_excel("Distance_Matrices/DistanceMatrix_2_aisles.xlsx", sheet_name = "DistanceMatrixMetres", header=None, skiprows= 1)
best_allocation, time_taken = LSH(Allocations.copy(), distance_matrix = distance_2_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Current Allocation with 2 middle aisle
Initial distance : 288474
Time taken to find neighbourhoods: 49.03 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 49.04 seconds.
Initial distance : 279354
Time taken to find neighbourhoods: 46.64 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 95.68 seconds.
Initial distance : 269862
Time taken to find neighbourhoods: 45.81 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 141.49 seconds.
Initial distance : 262392
Time taken to find neighbourhoods: 45.52 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 187.01 seconds.
Initial distance : 258588
Time taken to find neighbourhoods: 45.60 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 232.61 seconds.
Initial distance : 254946
Time taken to find neighbourhoods: 44.04 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 276.65 seconds.
Initial distance : 252102
Time taken to find neighbourhoods: 43.68 seconds.
===== VNS iteratio

In [126]:
# 3 middle aisle - current allocation
print("Results for LSH on the Current Allocation with 3 middle aisle")
distance_3_aisle = pd.read_excel("Distance_Matrices/DistanceMatrix_3_aisles.xlsx", sheet_name = "DistanceMatrixMetres", header=None, skiprows= 1)
best_allocation, time_taken = LSH(Allocations.copy(), distance_matrix = distance_3_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Current Allocation with 3 middle aisle
Initial distance : 294822
Time taken to find neighbourhoods: 45.83 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 45.83 seconds.
Initial distance : 285384
Time taken to find neighbourhoods: 46.79 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 92.62 seconds.
Initial distance : 279072
Time taken to find neighbourhoods: 46.07 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 138.69 seconds.
Initial distance : 267816
Time taken to find neighbourhoods: 45.94 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 184.63 seconds.
Initial distance : 263502
Time taken to find neighbourhoods: 46.19 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 230.82 seconds.
Initial distance : 260376
Time taken to find neighbourhoods: 46.44 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 277.27 seconds.
Initial distance : 257370
Time taken to find neighbourhoods: 46.32 seconds.
===== VNS iteratio

In [127]:
# 4 middle aisle - current allocation
print("Results for LSH on the Current Allocation with 4 middle aisle")
distance_4_aisle = pd.read_excel("Distance_Matrices/DistanceMatrix_4_aisles.xlsx", sheet_name = "DistanceMatrixMetres", header=None, skiprows= 1)
best_allocation, time_taken = LSH(Allocations.copy(), distance_matrix = distance_4_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the Current Allocation with 4 middle aisle
Initial distance : 303012
Time taken to find neighbourhoods: 44.48 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 44.48 seconds.
Initial distance : 293088
Time taken to find neighbourhoods: 44.29 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 88.77 seconds.
Initial distance : 282612
Time taken to find neighbourhoods: 43.72 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 132.49 seconds.
Initial distance : 274560
Time taken to find neighbourhoods: 43.52 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 176.01 seconds.
Initial distance : 270144
Time taken to find neighbourhoods: 43.11 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 219.12 seconds.
Initial distance : 266322
Time taken to find neighbourhoods: 44.24 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 263.36 seconds.
Initial distance : 262626
Time taken to find neighbourhoods: 45.35 seconds.
===== VNS iteratio

## Using most common product heuristic 

### 6 extra shelves filled

In [135]:
MCP_allocation_6extra_1mid = allocation_from_txt("Allocation_Vectors/CommonOrder_6extra_1mid_aisle.txt")
MCP_allocation_6extra_2mid = allocation_from_txt("Allocation_Vectors/CommonOrder_6extra_2mid_aisle.txt")
MCP_allocation_6extra_3mid = allocation_from_txt("Allocation_Vectors/CommonOrder_6extra_3mid_aisle.txt")
MCP_allocation_6extra_4mid = allocation_from_txt("Allocation_Vectors/CommonOrder_6extra_4mid_aisle.txt")

In [129]:
# 1 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 6 shelves filled and 1 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_6extra_1mid, distance_matrix = distance_1_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 1 middle aisle
Initial distance : 185976
Time taken to find neighbourhoods: 130.79 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 130.80 seconds.
Initial distance : 185706
Time taken to find neighbourhoods: 152.39 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 283.21 seconds.
Initial distance : 185532
Time taken to find neighbourhoods: 90.45 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 373.66 seconds.
Initial distance : 185394
Time taken to find neighbourhoods: 86.19 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 459.85 seconds.
Initial distance : 185082
Time taken to find neighbourhoods: 70.54 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 530.39 seconds.
Initial distance : 185010
Time taken to find neighbourhoods: 72.29 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 602.68 seconds.
Initial distance : 184986
Time taken to find neighbourhoods: 70.88 secon

In [136]:
# 2 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 6 shelves filled and 2 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_6extra_2mid, distance_matrix = distance_2_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 2 middle aisle


Initial distance : 186036
Time taken to find neighbourhoods: 171.67 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 171.67 seconds.
Initial distance : 185724
Time taken to find neighbourhoods: 170.93 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 342.61 seconds.
Initial distance : 185442
Time taken to find neighbourhoods: 178.53 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 521.15 seconds.
Initial distance : 185262
Time taken to find neighbourhoods: 210.66 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 731.81 seconds.
Initial distance : 185076
Time taken to find neighbourhoods: 229.73 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 961.56 seconds.
Initial distance : 184992
Time taken to find neighbourhoods: 254.15 seconds.
Total time taken: 1215.72 seconds.


In [149]:
# 3 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 6 shelves filled and 3 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_6extra_3mid, distance_matrix = distance_3_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 3 middle aisle
Initial distance : 189282
[(1, 24), 74, 29, 69, 57]
[1, 74, 29, 69, 57]
[22, (26, 95), 28, 84, 74]
[22, 26, 28, 84, 74]
[58, 7, 50, 92, (2, 71)]
[58, 7, 50, 92, 71]
[(2, 71), 51, 89, 44, 36]
[2, 51, 89, 44, 36]
Time taken to find neighbourhoods: 187.09 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 187.10 seconds.
Initial distance : 189066
[5, 45, 57, (26, 95), 50]
[5, 45, 57, 26, 50]
[58, 7, 50, 92, (2, 71)]
[58, 7, 50, 92, 71]
[70, (1, 24), 5, 27, 75]
[70, 1, 5, 27, 75]
[(2, 71), 51, 89, 44, 36]
[2, 51, 89, 44, 36]
[45, (25, 48), 51, 41, 5]
[45, 25, 51, 41, 5]
Time taken to find neighbourhoods: 126.30 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 313.39 seconds.
Initial distance : 188850
[5, 45, 57, (26, 95), 50]
[5, 45, 57, 26, 50]
[(1, 24), 5, 51, 69, 57]
[1, 5, 51, 69, 57]
[58, 7, 50, 92, (2, 71)]
[58, 7, 50, 92, 71]
[70, (1, 24), 5, 27, 75]
[70, 1, 5, 27, 75]
[35, 79, 46, (2, 71

In [153]:
# 4 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 6 shelves filled and 4 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_6extra_4mid, distance_matrix = distance_4_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 4 middle aisle
Initial distance : 192114
Time taken to find neighbourhoods: 121.87 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 121.88 seconds.
Initial distance : 191844
Time taken to find neighbourhoods: 150.43 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 272.31 seconds.
Initial distance : 191694
Time taken to find neighbourhoods: 140.00 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 412.31 seconds.
Initial distance : 191640
Time taken to find neighbourhoods: 132.09 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 544.41 seconds.
Initial distance : 191604
Time taken to find neighbourhoods: 153.72 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 698.13 seconds.
Initial distance : 191556
Time taken to find neighbourhoods: 129.51 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 827.64 seconds.
Initial distance : 191526
Time taken to find neighbourhoods: 127.44 

### 0 extra shelves filled

In [154]:
MCP_allocation_0extra_1mid = allocation_from_txt("Allocation_Vectors/CommonOrder_0extra_1mid_aisle.txt")
MCP_allocation_0extra_2mid = allocation_from_txt("Allocation_Vectors/CommonOrder_0extra_2mid_aisle.txt")
MCP_allocation_0extra_3mid = allocation_from_txt("Allocation_Vectors/CommonOrder_0extra_3mid_aisle.txt")
MCP_allocation_0extra_4mid = allocation_from_txt("Allocation_Vectors/CommonOrder_0extra_4mid_aisle.txt")

In [155]:
# 1 middle aisle - MCP allocation with 0 shelves
print("Results for LSH on the MCP allocations with 0 shelves filled and 1 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_0extra_1mid, distance_matrix = distance_1_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 1 middle aisle
Initial distance : 186138
Time taken to find neighbourhoods: 91.56 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 91.56 seconds.
Initial distance : 185826
Time taken to find neighbourhoods: 59.63 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 151.20 seconds.
Initial distance : 185724
Time taken to find neighbourhoods: 73.59 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 224.79 seconds.
Initial distance : 185424
Time taken to find neighbourhoods: 82.57 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 307.35 seconds.
Initial distance : 185232
Time taken to find neighbourhoods: 71.43 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 378.78 seconds.
Initial distance : 185106
Time taken to find neighbourhoods: 72.42 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 451.20 seconds.
Initial distance : 184974
Time taken to find neighbourhoods: 75.88 seconds.

In [156]:
# 2 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 0 shelves filled and 2 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_0extra_2mid, distance_matrix = distance_2_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 2 middle aisle


Initial distance : 186066
Time taken to find neighbourhoods: 74.01 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 74.02 seconds.
Initial distance : 185754
Time taken to find neighbourhoods: 61.91 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 135.93 seconds.
Initial distance : 185460
Time taken to find neighbourhoods: 65.84 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 201.77 seconds.
Initial distance : 185280
Time taken to find neighbourhoods: 65.45 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 267.22 seconds.
Initial distance : 185094
Time taken to find neighbourhoods: 63.76 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 330.99 seconds.
Initial distance : 184974
Time taken to find neighbourhoods: 62.94 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 393.93 seconds.
Initial distance : 184896
Time taken to find neighbourhoods: 72.87 seconds.
===== VNS iteration 7 =====
Time elapsed so far: 466.80 seconds.
Initial distan

In [157]:
# 3 middle aisle - MCP allocation with 6 shelves
print("Results for LSH on the MCP allocations with 0 shelves filled and 3 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_0extra_3mid, distance_matrix = distance_3_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 6 shelves filled and 3 middle aisle
Initial distance : 189336
Time taken to find neighbourhoods: 70.83 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 70.83 seconds.
Initial distance : 189096
Time taken to find neighbourhoods: 130.28 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 201.14 seconds.
Initial distance : 188874
Time taken to find neighbourhoods: 79.99 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 281.14 seconds.
Initial distance : 188700
Time taken to find neighbourhoods: 79.27 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 360.41 seconds.
Initial distance : 188640
Time taken to find neighbourhoods: 76.42 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 436.83 seconds.
Initial distance : 188526
Time taken to find neighbourhoods: 90.37 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 527.21 seconds.
Initial distance : 188490
Time taken to find neighbourhoods: 84.11 seconds

In [158]:
# 4 middle aisle - MCP allocation with 0 shelves
print("Results for LSH on the MCP allocations with 0 shelves filled and 4 middle aisle")
best_allocation, time_taken = LSH(MCP_allocation_0extra_4mid, distance_matrix = distance_4_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the MCP allocations with 0 shelves filled and 4 middle aisle
Initial distance : 192162
Time taken to find neighbourhoods: 109.83 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 109.84 seconds.
Initial distance : 191880
Time taken to find neighbourhoods: 87.62 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 197.46 seconds.
Initial distance : 191604
Time taken to find neighbourhoods: 101.37 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 298.83 seconds.
Initial distance : 191568
Time taken to find neighbourhoods: 84.95 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 383.78 seconds.
Initial distance : 191478
Time taken to find neighbourhoods: 80.61 seconds.
Total time taken: 464.39 seconds.


### Greedy heuristic allocation

In [164]:
GH_1mid = allocation_from_txt("Allocation_Vectors/Greedy_1mid_aisle.txt")
GH_2mid = allocation_from_txt("Allocation_Vectors/Greedy_2mid_aisle.txt")
GH_3mid = allocation_from_txt("Allocation_Vectors/Greedy_3mid_aisle.txt")
GH_4mid = allocation_from_txt("Allocation_Vectors/Greedy_4mid_aisle.txt")

In [165]:
# 1 middle aisle - GH allocation 
print("Results for LSH on the GH allocation and 1 middle aisle")
best_allocation, time_taken = LSH(GH_1mid, distance_matrix = distance_1_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the GH allocation and 1 middle aisle
Initial distance : 185514
Time taken to find neighbourhoods: 110.73 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 110.73 seconds.
Initial distance : 185106
Time taken to find neighbourhoods: 85.66 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 196.39 seconds.
Initial distance : 184914
Time taken to find neighbourhoods: 134.20 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 330.59 seconds.
Initial distance : 184848
Time taken to find neighbourhoods: 72.95 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 403.55 seconds.
Initial distance : 184794
Time taken to find neighbourhoods: 66.47 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 470.01 seconds.
Initial distance : 184596
Time taken to find neighbourhoods: 68.79 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 538.81 seconds.
Initial distance : 184560
Time taken to find neighbourhoods: 69.85 seconds.
===== VNS iteration 

In [169]:
# 2 middle aisle - GH allocation 
print("Results for LSH on the GH allocation and 2 middle aisle")
best_allocation, time_taken = LSH(GH_2mid, distance_matrix = distance_2_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the GH allocation and 2 middle aisle
Initial distance : 185472
Time taken to find neighbourhoods: 91.82 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 91.82 seconds.
Initial distance : 185292
Time taken to find neighbourhoods: 173.97 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 265.80 seconds.
Initial distance : 185136
Time taken to find neighbourhoods: 161.04 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 426.84 seconds.
Initial distance : 185022
Time taken to find neighbourhoods: 130.53 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 557.37 seconds.
Initial distance : 184944
Time taken to find neighbourhoods: 114.94 seconds.
===== VNS iteration 5 =====
Time elapsed so far: 672.31 seconds.
Initial distance : 184902
Time taken to find neighbourhoods: 125.05 seconds.
===== VNS iteration 6 =====
Time elapsed so far: 797.36 seconds.
Initial distance : 184758
Time taken to find neighbourhoods: 123.14 seconds.
===== VNS iterati

In [174]:
# 3 middle aisle - GH allocation 
print("Results for LSH on the GH allocation and 3 middle aisle")
best_allocation, time_taken = LSH(GH_3mid, distance_matrix = distance_3_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the GH allocation and 3 middle aisle
Initial distance : 188088
Time taken to find neighbourhoods: 121.14 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 121.14 seconds.
Initial distance : 187926
Time taken to find neighbourhoods: 121.17 seconds.
===== VNS iteration 2 =====
Time elapsed so far: 242.31 seconds.
Initial distance : 187896
Time taken to find neighbourhoods: 126.12 seconds.
===== VNS iteration 3 =====
Time elapsed so far: 368.43 seconds.
Initial distance : 187884
Time taken to find neighbourhoods: 109.57 seconds.
===== VNS iteration 4 =====
Time elapsed so far: 478.00 seconds.
Initial distance : 187638
Time taken to find neighbourhoods: 123.68 seconds.
Total time taken: 601.67 seconds.


In [172]:
# 4 middle aisle - GH allocation 
print("Results for LSH on the GH allocation and 4 middle aisle")
best_allocation, time_taken = LSH(GH_4mid, distance_matrix = distance_4_aisle, max_k=5, time_limit=3600)
print(f"Total time taken: {time_taken:.2f} seconds.")

Results for LSH on the GH allocation and 4 middle aisle
Initial distance : 191106
Time taken to find neighbourhoods: 102.24 seconds.
===== VNS iteration 1 =====
Time elapsed so far: 102.24 seconds.
Initial distance : 190536
Time taken to find neighbourhoods: 93.92 seconds.
Total time taken: 196.16 seconds.
